In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# add your code here - consider creating a new cell for each section of code
# Filter users with at least 200 ratings
user_counts = df_ratings['user'].value_counts()
filtered_users = user_counts[user_counts >= 200].index

# Filter books with at least 100 ratings
book_counts = df_ratings['isbn'].value_counts()
filtered_books = book_counts[book_counts >= 100].index

# Filter ratings dataframe accordingly
ratings_filtered = df_ratings[
    (df_ratings['user'].isin(filtered_users)) &
    (df_ratings['isbn'].isin(filtered_books))
]


In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book=""):
    # Create pivot table: books x users, fill NaNs with 0
    book_user_matrix = ratings_filtered.pivot(index='isbn', columns='user', values='rating').fillna(0)

    # Sparse matrix for KNN
    book_user_sparse = csr_matrix(book_user_matrix.values)

    # Fit KNN model with cosine metric
    knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
    knn_model.fit(book_user_sparse)

    # Maps to convert isbn <-> title
    isbn_to_title = dict(zip(df_books['isbn'], df_books['title']))
    title_to_isbn = dict(zip(df_books['title'], df_books['isbn']))

    # Check if book exists
    if book not in title_to_isbn:
        return f"Book '{book}' not found."

    book_isbn = title_to_isbn[book]

    # Check if book is in filtered matrix
    if book_isbn not in book_user_matrix.index:
        return f"Book '{book}' does not have enough ratings."

    # Find index of the book in pivot table
    book_idx = book_user_matrix.index.get_loc(book_isbn)

    # Get 6 neighbors (including itself)
    distances, indices = knn_model.kneighbors(book_user_sparse[book_idx], n_neighbors=6)

    # Build recommendations list, skipping the first (the book itself)
    recommended_books = []
    for i in range(1, 6):
        idx = indices[0][i]
        dist = distances[0][i]
        rec_isbn = book_user_matrix.index[idx]
        rec_title = isbn_to_title.get(rec_isbn, "Unknown Title")
        recommended_books.append([rec_title, dist])

    return [book, recommended_books]


In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()